<a href="https://colab.research.google.com/github/worldterminator/mess/blob/main/leap.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Rationale

For each year, I keep only the matched records from enrollment and assessment (tho score could be NA) files. Yet across different years, I keep the unmatched student IDs.
Then, each student that remains in the sample should have scores in all their enrolled years (NA included), allowing me to identify cohorts based on first year of enrollment and grade levels.

#Attempt with 16-17 data

In [62]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [63]:
import pandas as pd

# File path
file_path = '/content/drive/My Drive/Assessment201617.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# Display the first few rows of the DataFrame
data.head()


,TermName,DistrictName,District_StateID,SchoolName,School_StateID,StudentID,Student_StateID,Subject,Course,GrowthMeasureYN,...,ProjectedProficiencyStudy6,ProjectedProficiencyLevel6,ProjectedProficiencyStudy7,ProjectedProficiencyLevel7,ProjectedProficiencyStudy8,ProjectedProficiencyLevel8,ProjectedProficiencyStudy9,ProjectedProficiencyLevel9,ProjectedProficiencyStudy10,ProjectedProficiencyLevel10
0,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1002,1591820143.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1002,1591820143.000,Language Arts,Reading,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1003,7816959131.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1003,7816959131.000,Language Arts,Reading,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Spring 2016-2017,Leap Academy University Charter School,807109,STEM Lower Elementary,NaN,1005,6200882279.000,Mathematics,Math K-12,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


List of column names you want to drop
columns_to_drop = ['column_name4', 'column_name5']

Drop these columns
data.drop(columns=columns_to_drop, inplace=True)

In [64]:
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1617= data[columns_to_keep]

score1617.head()


,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,213,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,195,511
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,196,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,183,295
4,1005,6200882279.000,Growth: Math K-2 CCSS 2010 V2,191,NaN


In [65]:
# Set display option to avoid scientific notation
pd.set_option('display.float_format', lambda x: '%.3f' % x)


In [66]:
print(score1617)

# if want to convert state ID: map1617['column_name'] = map1617['column_name'].astype(int)


       StudentID  Student_StateID                          TestName  \
0           1002   1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002   1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003   7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003   7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005   6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...              ...                               ...   
3066  9982194839              NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412   9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412   9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608              NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608              NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  
0              213         NaN  
1              195         511  
2              196         NaN  
3              

Current strategy: join RIT score with enrollment for each year, and then making panels

In [67]:
# File path
file_path = '/content/drive/My Drive/Enrollment201617.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# Display the first few rows of the DataFrame
data.head()

,TermName,DistrictName,District_StateID,SchoolName,School_StateID,StudentID,Student_StateID,StudentDateOfBirth,StudentEthnicGroup,NWEAStandard_EthnicGroup,StudentGender,Grade,NWEAStandard_Grade
0,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1018256150,1018256150.000,7/22/2000,Hispanic,Hispanic or Latino,M,11,11
1,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1084712619,NaN,12/6/1998,Hispanic,Hispanic or Latino,M,12,12
2,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1163227042,NaN,10/31/2000,Hispanic,Hispanic or Latino,F,10,10
3,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,1183257367,NaN,3/19/1999,Black,Black or African American,M,12,12
4,Spring 2016-2017,Leap Academy University Charter School,807109,STEAM High School,NaN,12272753,6136857128.000,2/29/2000,Hispanic,Hispanic or Latino,F,11,11


In [68]:
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1617= data[columns_to_keep]

enrollment1617.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1018256150,1018256150.000,Hispanic,M,11
1,1084712619,NaN,Hispanic,M,12
2,1163227042,NaN,Hispanic,F,10
3,1183257367,NaN,Black,M,12
4,12272753,6136857128.000,Hispanic,F,11


In [69]:

# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1617, enrollment1617, on='StudentID', how='inner', indicator=True)

merged16 = merged_df[merged_df['_merge'] == 'both']

merged16.drop(columns=['_merge'], inplace=True)

tried merging on one of 'student id' and 'state id' sperately and combine the merges and hadnle teh duplicate columns, did not work out well bc multiple observations matched to the same id/combination of ids? not sure, but it produced 100,000 entries

In [70]:
print(merged16)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005     6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3066  9982194839                NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608                NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              213         NaN     1591820143.000  

In [71]:
# Add a new column named 'Year' with all values set to 2016
merged16['Year'] = 2016

# Display the DataFrame to see the new column
print(merged16)


       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1005     6200882279.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3066  9982194839                NaN   Growth: Reading 6+ CCSS 2010 V2   
3067  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
3068  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   
3069  9983668608                NaN     Growth: Math 2-5 CCSS 2010 V2   
3070  9983668608                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              213         NaN     1591820143.000  

In [116]:
print(merged16.dtypes)

StudentID               int64
Student_StateID_x     float64
TestName               object
TestRITScore            int64
LexileScore            object
Student_StateID_y     float64
StudentEthnicGroup     object
StudentGender          object
Grade                  object
Year                    int64
dtype: object


#14-15 data

In [72]:
# File path
file_path = '/content/drive/My Drive/assessments201415.csv'

data= pd.read_csv(file_path)
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1415= data[columns_to_keep]

score1415.head()


,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,170,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,169,BR
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,154,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,157,BR
4,1008,4297133420.000,Growth: Math K-2 CCSS 2010 V2,157,NaN


In [73]:
# File path
file_path = '/content/drive/My Drive/enrollment201415.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1415= data[columns_to_keep]

enrollment1415.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1015267642,1015267642.000,Hispanic,M,6
1,1023348629,NaN,Asian,F,8
2,10483,NaN,Black,F,7
3,10630,NaN,Black,F,6
4,10679,NaN,Hispanic,F,6


In [74]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1415, enrollment1415, on='StudentID', how='inner', indicator=True)

merged14 = merged_df[merged_df['_merge'] == 'both']

merged14.drop(columns=['_merge'], inplace=True)

print(merged14)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1008     4297133420.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3180  9974474112                NaN      Growth: Math 6+ CCSS 2010 V2   
3181  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3182  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3183  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
3184  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              170         NaN     1591820143.000  

In [75]:
# Add a new column named 'Year' with all values set to 2016
merged14['Year'] = 2014

# Display the DataFrame to see the new column
print(merged14)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1008     4297133420.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
3180  9974474112                NaN      Growth: Math 6+ CCSS 2010 V2   
3181  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3182  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
3183  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
3184  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              170         NaN     1591820143.000  

In [115]:
print(merged14.dtypes)

StudentID              object
Student_StateID_x     float64
TestName               object
TestRITScore            int64
LexileScore            object
Student_StateID_y     float64
StudentEthnicGroup     object
StudentGender          object
Grade                  object
Year                    int64
dtype: object


In [119]:
# Identify non-numeric values
merged14['StudentID_is_numeric'] = pd.to_numeric(merged14['StudentID'], errors='coerce').notna()

# Option 1: Keep only rows where StudentID is numeric
merged14 = merged14[merged14['StudentID_is_numeric']]

# Convert to integer
merged14['StudentID'] = merged14['StudentID'].astype(int)

# Check the change
print(merged14.dtypes)


StudentID                 int64
Student_StateID_x       float64
TestName                 object
TestRITScore              int64
LexileScore              object
Student_StateID_y       float64
StudentEthnicGroup       object
StudentGender            object
Grade                    object
Year                      int64
StudentID_is_numeric       bool
dtype: object


<ipython-input-119-ff2ba08e3169>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged14['StudentID'] = merged14['StudentID'].astype(int)


In [120]:
merged14['StudentID'] = merged14['StudentID'].astype(int)
# Check the change
print(merged14.dtypes)

StudentID                 int64
Student_StateID_x       float64
TestName                 object
TestRITScore              int64
LexileScore              object
Student_StateID_y       float64
StudentEthnicGroup       object
StudentGender            object
Grade                    object
Year                      int64
StudentID_is_numeric       bool
dtype: object


# 15-16 data

In [76]:
# File path
file_path = '/content/drive/My Drive/assessments201516.csv'

data= pd.read_csv(file_path)
# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']

# Create a new DataFrame with only these columns
score1516= data[columns_to_keep]

score1516.head()

,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1002,1591820143.000,Growth: Math K-2 CCSS 2010 V2,195,NaN
1,1002,1591820143.000,Growth: Reading K-2 CCSS 2010,187,367
2,1003,7816959131.000,Growth: Math K-2 CCSS 2010 V2,167,NaN
3,1003,7816959131.000,Growth: Reading K-2 CCSS 2010,167,BR
4,1011,8425580900.000,Growth: Math K-2 CCSS 2010 V2,185,NaN


In [77]:
# File path
file_path = '/content/drive/My Drive/enrollment201516.csv'

# Read the CSV file
data= pd.read_csv(file_path)

# List of column names you want to keep
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']

# Create a new DataFrame with only these columns
enrollment1516= data[columns_to_keep]

enrollment1516.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1018256150,1018256150.000,Hispanic,M,10
1,1046427506,NaN,Hispanic,F,12
2,1084712619,NaN,Hispanic,M,11
3,1088668664,NaN,Hispanic,M,12
4,1121380680,NaN,Hispanic,F,12


In [78]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1516, enrollment1516, on='StudentID', how='inner', indicator=True)

merged15 = merged_df[merged_df['_merge'] == 'both']

merged15.drop(columns=['_merge'], inplace=True)

print(merged15)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1011     8425580900.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
2899  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
2900  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
2901  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   
2902  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
2903  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              195         NaN     1591820143.000  

In [79]:
# Add a new column named 'Year' with all values set to 2016
merged15['Year'] = 2015

# Display the DataFrame to see the new column
print(merged15)

       StudentID  Student_StateID_x                          TestName  \
0           1002     1591820143.000     Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000     Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000     Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000     Growth: Reading K-2 CCSS 2010   
4           1011     8425580900.000     Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                               ...   
2899  9974474112                NaN   Growth: Reading 6+ CCSS 2010 V2   
2900  9980446488                NaN     Growth: Math 2-5 CCSS 2010 V2   
2901  9980446488                NaN  Growth: Reading 2-5 CCSS 2010 V2   
2902  9983529412     9983529412.000     Growth: Math 2-5 CCSS 2010 V2   
2903  9983529412     9983529412.000  Growth: Reading 2-5 CCSS 2010 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              195         NaN     1591820143.000  

# 17-18 data

In [80]:
file_path = '/content/drive/My Drive/assessments201718.csv'
data= pd.read_csv(file_path)
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']
score1718= data[columns_to_keep]
score1718.head()

,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1001,9784372138,Growth: Math 2-5 NJ 2016,175,NaN
1,1001,9784372138,Growth: Reading 2-5 NJ 2016,168,BR
2,1002,1591820143,Growth: Math 2-5 NJ 2016,211,NaN
3,1002,1591820143,Growth: Reading 2-5 NJ 2016,207,727
4,1003,7816959131,Growth: Math 2-5 NJ 2016,194,NaN


In [81]:
file_path = '/content/drive/My Drive/enrollment201718.csv'
data= pd.read_csv(file_path)
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']
enrollment1718= data[columns_to_keep]
enrollment1718.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,1015267642,1015267642,Hispanic,M,9
1,1018256150,1018256150,Hispanic,M,12
2,12272794,8032207693,Black,F,11
3,12272797,1325317881,Black,M,11
4,12272798,1021430407,Black,F,11


In [82]:
# Merge on 'StudentID' with an indicator
merged_df = pd.merge(score1718, enrollment1718, on='StudentID', how='inner', indicator=True)

merged17 = merged_df[merged_df['_merge'] == 'both']

merged17.drop(columns=['_merge'], inplace=True)

print(merged17)

       StudentID  Student_StateID_x                     TestName  \
0           1001         9784372138     Growth: Math 2-5 NJ 2016   
1           1001         9784372138  Growth: Reading 2-5 NJ 2016   
2           1002         1591820143     Growth: Math 2-5 NJ 2016   
3           1002         1591820143  Growth: Reading 2-5 NJ 2016   
4           1003         7816959131     Growth: Math 2-5 NJ 2016   
...          ...                ...                          ...   
3006         994         3356386762  Growth: Reading 2-5 NJ 2016   
3007         996         7232964515     Growth: Math 2-5 NJ 2016   
3008         996         7232964515  Growth: Reading 2-5 NJ 2016   
3009  9977980915         9977980915     Growth: Math K-2 NJ 2016   
3010  9977980915         9977980915  Growth: Reading K-2 NJ 2016   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              175         NaN         9784372138              Black   
1              168          BR         

In [83]:
# Add a new column named 'Year' with all values set to 2017
merged17['Year'] = 2017
print(merged17)

       StudentID  Student_StateID_x                     TestName  \
0           1001         9784372138     Growth: Math 2-5 NJ 2016   
1           1001         9784372138  Growth: Reading 2-5 NJ 2016   
2           1002         1591820143     Growth: Math 2-5 NJ 2016   
3           1002         1591820143  Growth: Reading 2-5 NJ 2016   
4           1003         7816959131     Growth: Math 2-5 NJ 2016   
...          ...                ...                          ...   
3006         994         3356386762  Growth: Reading 2-5 NJ 2016   
3007         996         7232964515     Growth: Math 2-5 NJ 2016   
3008         996         7232964515  Growth: Reading 2-5 NJ 2016   
3009  9977980915         9977980915     Growth: Math K-2 NJ 2016   
3010  9977980915         9977980915  Growth: Reading K-2 NJ 2016   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              175         NaN         9784372138              Black   
1              168          BR         

#18-19 data

In [84]:
file_path = '/content/drive/My Drive/assessments201819.csv'
data= pd.read_csv(file_path)
columns_to_keep = ['StudentID', 'Student_StateID', 'TestName','TestRITScore','LexileScore']
score1819= data[columns_to_keep]
score1819.head()

,StudentID,Student_StateID,TestName,TestRITScore,LexileScore
0,1001,9784372138,Growth: Math 2-5 NJ 2016,199,NaN
1,1001,9784372138,Growth: Reading 2-5 NJ 2016 V2,201,685L
2,1002,1591820143,Growth: Math 2-5 NJ 2016,231,NaN
3,1002,1591820143,Growth: Reading 2-5 NJ 2016 V2,223,1130L
4,1003,7816959131,Growth: Math 2-5 NJ 2016,205,NaN


In [85]:
file_path = '/content/drive/My Drive/enrollment201819.csv'
data= pd.read_csv(file_path)
columns_to_keep = ['StudentID', 'Student_StateID', 'StudentEthnicGroup','StudentGender','Grade']
enrollment1819= data[columns_to_keep]
enrollment1819.head()

,StudentID,Student_StateID,StudentEthnicGroup,StudentGender,Grade
0,12272794,8032207693,Black,F,12
1,12272797,1325317881,Black,M,12
2,12272798,1021430407,Black,F,12
3,12272799,1312577972,Hispanic,M,12
4,12272801,2047487493,Hispanic,F,12


In [86]:
merged_df = pd.merge(score1819, enrollment1819, on='StudentID', how='inner', indicator=True)
merged18 = merged_df[merged_df['_merge'] == 'both']
merged18.drop(columns=['_merge'], inplace=True)
print(merged18)

      StudentID  Student_StateID_x                        TestName  \
0          1001         9784372138        Growth: Math 2-5 NJ 2016   
1          1001         9784372138  Growth: Reading 2-5 NJ 2016 V2   
2          1002         1591820143        Growth: Math 2-5 NJ 2016   
3          1002         1591820143  Growth: Reading 2-5 NJ 2016 V2   
4          1003         7816959131        Growth: Math 2-5 NJ 2016   
...         ...                ...                             ...   
2756        996         7232964515  Growth: Reading 2-5 NJ 2016 V2   
2757        997         3224073437        Growth: Math 2-5 NJ 2016   
2758        997         3224073437  Growth: Reading 2-5 NJ 2016 V2   
2759        999         6896858008        Growth: Math 2-5 NJ 2016   
2760        999         6896858008  Growth: Reading 2-5 NJ 2016 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              199         NaN         9784372138              Black   
1              

In [87]:
merged18['Year'] = 2018
print(merged18)

      StudentID  Student_StateID_x                        TestName  \
0          1001         9784372138        Growth: Math 2-5 NJ 2016   
1          1001         9784372138  Growth: Reading 2-5 NJ 2016 V2   
2          1002         1591820143        Growth: Math 2-5 NJ 2016   
3          1002         1591820143  Growth: Reading 2-5 NJ 2016 V2   
4          1003         7816959131        Growth: Math 2-5 NJ 2016   
...         ...                ...                             ...   
2756        996         7232964515  Growth: Reading 2-5 NJ 2016 V2   
2757        997         3224073437        Growth: Math 2-5 NJ 2016   
2758        997         3224073437  Growth: Reading 2-5 NJ 2016 V2   
2759        999         6896858008        Growth: Math 2-5 NJ 2016   
2760        999         6896858008  Growth: Reading 2-5 NJ 2016 V2   

      TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0              199         NaN         9784372138              Black   
1              

# Merge all data

In [121]:
precovid = pd.concat([merged14, merged15, merged16, merged17, merged18], ignore_index = True)

In [122]:
YearEnrolled = precovid.groupby('StudentID')['Year'].agg(['min', 'max'])
YearEnrolled.rename(columns={'min': 'first_year', 'max': 'last_year'}, inplace=True)

In [124]:
# Assuming df is your DataFrame and student_id is the ID you're looking for
StudentID = 1002  # replace with the actual StudentID
row = YearEnrolled.loc[StudentID]
print(row)
#interesting, so 2014 record isn't in

first_year    2014
last_year     2018
Name: 1002, dtype: int64


In [125]:
precovid = precovid.merge(YearEnrolled, on='StudentID', how='left')

In [126]:
precovid['Duration'] = precovid['last_year'] - precovid['first_year']

In [127]:
print(precovid)

       StudentID  Student_StateID_x                        TestName  \
0           1002     1591820143.000   Growth: Math K-2 CCSS 2010 V2   
1           1002     1591820143.000   Growth: Reading K-2 CCSS 2010   
2           1003     7816959131.000   Growth: Math K-2 CCSS 2010 V2   
3           1003     7816959131.000   Growth: Reading K-2 CCSS 2010   
4           1008     4297133420.000   Growth: Math K-2 CCSS 2010 V2   
...          ...                ...                             ...   
14908        996     7232964515.000  Growth: Reading 2-5 NJ 2016 V2   
14909        997     3224073437.000        Growth: Math 2-5 NJ 2016   
14910        997     3224073437.000  Growth: Reading 2-5 NJ 2016 V2   
14911        999     6896858008.000        Growth: Math 2-5 NJ 2016   
14912        999     6896858008.000  Growth: Reading 2-5 NJ 2016 V2   

       TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
0               170         NaN     1591820143.000           Hispanic   
1

<ipython-input-127-e8abc804599d>:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  print(precovid)


In [128]:
summary_stats = precovid['Duration'].describe()
print(summary_stats)

count   14913.000
mean        2.587
std         1.354
min         0.000
25%         2.000
50%         3.000
75%         4.000
max         4.000
Name: Duration, dtype: float64


In [129]:
precovid['Duration'] = precovid['Duration'] + 1
print(precovid.head())

   StudentID  Student_StateID_x                       TestName  TestRITScore  \
0       1002     1591820143.000  Growth: Math K-2 CCSS 2010 V2           170   
1       1002     1591820143.000  Growth: Reading K-2 CCSS 2010           169   
2       1003     7816959131.000  Growth: Math K-2 CCSS 2010 V2           154   
3       1003     7816959131.000  Growth: Reading K-2 CCSS 2010           157   
4       1008     4297133420.000  Growth: Math K-2 CCSS 2010 V2           157   

  LexileScore  Student_StateID_y StudentEthnicGroup StudentGender Grade  Year  \
0         NaN     1591820143.000           Hispanic             M    KF  2014   
1          BR     1591820143.000           Hispanic             M    KF  2014   
2         NaN     7816959131.000              Black             M    KF  2014   
3          BR     7816959131.000              Black             M    KF  2014   
4         NaN     4297133420.000           Hispanic             M    KF  2014   

  StudentID_is_numeric  first_ye

# cohort (from KF to G4) vs later to join

In [145]:
# Filter the main DataFrame for the year 2018
outcome2018 = precovid[precovid['Year'] == 2018]

# Continuous Enrollment Cohort
continuous_enrollment_cohort = outcome2018[(outcome2018['Duration'] == 5) & (outcome2018['last_year'] == 2018)]

# Late Joiners (joined later but reached Grade 4 in 2018)
late_joiners = outcome2018[(outcome2018['Duration'] < 5) & (outcome2018['last_year'] == 2018)& (outcome2018['Grade'] == '4')]


In [146]:
print(late_joiners)

       StudentID  Student_StateID_x                        TestName  \
12152       1001     9784372138.000        Growth: Math 2-5 NJ 2016   
12153       1001     9784372138.000  Growth: Reading 2-5 NJ 2016 V2   
12158       1005     6200882279.000        Growth: Math 2-5 NJ 2016   
12159       1005     6200882279.000  Growth: Reading 2-5 NJ 2016 V2   
12162       1009     2498295332.000        Growth: Math 2-5 NJ 2016   
...          ...                ...                             ...   
14908        996     7232964515.000  Growth: Reading 2-5 NJ 2016 V2   
14909        997     3224073437.000        Growth: Math 2-5 NJ 2016   
14910        997     3224073437.000  Growth: Reading 2-5 NJ 2016 V2   
14911        999     6896858008.000        Growth: Math 2-5 NJ 2016   
14912        999     6896858008.000  Growth: Reading 2-5 NJ 2016 V2   

       TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
12152           199         NaN     9784372138.000              Black   
1

In [136]:
print(continuous_enrollment_cohort)

       StudentID  Student_StateID_x                        TestName  \
12154       1002     1591820143.000        Growth: Math 2-5 NJ 2016   
12155       1002     1591820143.000  Growth: Reading 2-5 NJ 2016 V2   
12156       1003     7816959131.000        Growth: Math 2-5 NJ 2016   
12157       1003     7816959131.000  Growth: Reading 2-5 NJ 2016 V2   
12160       1008     4297133420.000        Growth: Math 2-5 NJ 2016   
...          ...                ...                             ...   
14898        989     4582807302.000  Growth: Reading 2-5 NJ 2016 V2   
14903        993     8137617860.000        Growth: Math 2-5 NJ 2016   
14904        993     8137617860.000  Growth: Reading 2-5 NJ 2016 V2   
14905        994     3356386762.000        Growth: Math 2-5 NJ 2016   
14906        994     3356386762.000  Growth: Reading 2-5 NJ 2016 V2   

       TestRITScore LexileScore  Student_StateID_y StudentEthnicGroup  \
12154           231         NaN     1591820143.000           Hispanic   
1

Welch's t-test was used, which does not assume equal variances between the two groups.

In [150]:
# Perform the t-test on Math TestRITScore
t_stat_math, p_value_math = stats.ttest_ind(
    continuous_enrollment_cohort[continuous_enrollment_cohort['TestName'].str.contains("Math")]['TestRITScore'].dropna(),
    late_joiners[late_joiners['TestName'].str.contains("Math")]['TestRITScore'].dropna(),
    equal_var=False
)

# Print the results for Math test
print("T-statistic for Math:", t_stat_math)
print("P-value for Math: {:.10f}".format(p_value_math))


T-statistic for Math: 7.540188637233863
P-value for Math: 0.0000000002


In [152]:
# Perform the t-test on Reading TestRITScore
t_stat_reading, p_value_reading = stats.ttest_ind(
    continuous_enrollment_cohort[continuous_enrollment_cohort['TestName'].str.contains("Reading")]['TestRITScore'].dropna(),
    late_joiners[late_joiners['TestName'].str.contains("Reading")]['TestRITScore'].dropna(),
    equal_var=False
)

# Print the results for Math test
print("T-statistic for Reading:", t_stat_reading)
print("P-value for Reading: {:.10f}".format(p_value_reading))

T-statistic for Reading: 6.771368131144291
P-value for Reading: 0.0000000065
